### Cognitive Modeling HW4 Problem 2

In [1]:
# Pip imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import stan

try:
    import arviz as az
except ImportError as err:
    print("Please, install arviz for easy visualization of Stan models.")

import nest_asyncio
nest_asyncio.apply()

import pandas as pd

print(stan.__version__)

/home/krupkaa/anaconda3/envs/cognitive/lib/python3.12/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


3.10.0


In [2]:
one_high_stan_code = """
data {
  int<lower=1> N; // Number of trials
  array[2] int<lower=0, upper=N> freqs;
}

parameters {
  // Your code here
  int<lower=0, higher=1> d;
  int<lower=0, higher=1> g;
}

transformed parameters {
  simplex[2] theta;
  
  // New
  theta[1] = a * t * f;
  // Neologism
  //theta[2] = a*t*(1-f)*(1-c) + a*(1-t)*(1-f)*(1-c); // NOT NEEDED DUE TO SIMPLIFICATION
  theta[2] = (a * (1-f) * (1-c))
  // Formal
  theta[3] = a*(t*(1-f)*c + (1-t)*(1-f)*c);
  // Mixed
  theta[4] = a*(1-t)*f;
  // Other
  theta[5] = 1-a;
}

model {
  // Assuming uniform priors
  target += beta_lpdf(a | 1, 1);
  target += beta_lpdf(t | 1, 1);
  target += beta_lpdf(f | 1, 1);
  target += beta_lpdf(c | 1, 1);

  // Multinomial likelihood
  target += multinomial_lpmf(freqs | theta);
}

generated quantities{
  array[K] int pred_freqs;
  pred_freqs = multinomial_rng(theta, N);
}
"""